In [ ]:
import os, sys, argparse, re, glob, csv, subprocess
import pandas as pd
import numpy as np
from datetime import datetime
import PyAstronomy
from PyAstronomy import pyasl
import datetime
from datetime import datetime, timedelta
pd.set_option('display.max_rows', 100)

os.chdir("")

## SEE the brill https://www.science.org/doi/full/10.1126/science.abf2946


deaths_alles=pd.read_csv("global_deaths.csv")

pop_size_df = pd.read_csv("population_stats_UN.csv")


latent_n_days =3
incubate_n_days= 5
infectious_n_days = 7
prop_asymptomatic = 0.31
IFR =100
inf_death_day = 23


In [14]:

##Calculate estimated proportion seeders

MT=pd.DataFrame()

inf_mat={}

for country in deaths_alles["location"].unique():

    deaths=deaths_alles[deaths_alles["location"]==country]
    pop_size=pop_size_df[pop_size_df["location"]==country]["population_size"].values[0]

    ##age_of_infection_matrix

    n_days=len(deaths["daily_deaths"].to_list())+inf_death_day

    ww=np.zeros((n_days, (inf_death_day+1)))
    inframe_deaths=[[0.0]*inf_death_day, deaths["daily_deaths"].to_list(),  [None]*inf_death_day]
    inframe_deaths=[sl for l in inframe_deaths for sl in l]

    for d in range(0, n_days, 1):
        w=[inframe_deaths[(d-1) + xx] for xx in (range(0, inf_death_day+1,1))]
        ww[d,]=w[::-1]
        inf_mat[country]=ww


    ##padded_potential_seeders

    presympt_cases = ww[:, [xx for xx in range(0, incubate_n_days + 1, 1)]]
    asympt_cases = prop_asymptomatic * ww[: ,[xx for xx in  range((incubate_n_days+1), ((latent_n_days + infectious_n_days) + 1), 1)]]
    tot_seed=np.sum(presympt_cases, axis = 1) + np.sum(asympt_cases, axis = 1)


    ##seeder_proportion

    start = datetime.strptime(deaths["date"].min(), '%Y-%m-%d')
    rstart=start- timedelta(days=inf_death_day)
    add_me=[(rstart + timedelta(days=x)).strftime("%Y-%m-%d") for x in range((start-rstart).days)]
    dd=[add_me, deaths["date"].to_list()]
    dd=[sl for l in dd for sl in l]
    dd=dd[:-1]
    
    seed_prop=IFR * tot_seed[1:] / pop_size

    fdf=pd.DataFrame(dd, seed_prop).reset_index().rename(columns={"index":"seeder_proportion", 0:"date"})
    fdf["country"]=country
    MT=MT.append(fdf)

    
MT.to_csv("proportion-seeders.csv", index=None)

In [15]:
##location_num_infections

MT2=pd.DataFrame()

for country in deaths_alles["location"].unique():
    deaths=deaths_alles[deaths_alles["location"]==country]
    nr_deaths_infs=inf_mat[country][:,1]

    start = datetime.strptime(deaths["date"].min(), '%Y-%m-%d')
    rstart=start- timedelta(days=inf_death_day)
    add_me=[(rstart + timedelta(days=x)).strftime("%Y-%m-%d") for x in range((start-rstart).days)]
    dd=[add_me, deaths["date"].to_list()]
    dd=[sl for l in dd for sl in l]
    
    
    
    nr_infect=IFR*nr_deaths_infs[2:]
    dd=dd[:len(nr_infect)]
    
    fdf2=pd.DataFrame(dd, nr_infect).reset_index().rename(columns={"index":"nr_infections", 0:"date"})
    fdf2["country"]=country
    MT2=MT2.append(fdf2)
    
MT2.to_csv("daily-infections.csv", index=None)

In [19]:
##travel data cannot be released without permission

arrive=pd.read_csv("arrivals_air.csv")


rd={"Antigua-Barbuda":"Antigua and Barbuda", "Bosnia-Herzegovina":"Bosnia and Herzegovina", 
    "Brunei Darussalam":"Brunei",
"Central African Rep":"Central African Republic", "Cote D'Ivoire":"Cote d'Ivoire", 
    "Dominican Rep":"Dominican Rep", 
"Korea (South)":"Korea, South", "Macedonia":"North Macedonia", 
    "St Kitts-Nevis":"Saint Kitts and Nevis", "Taiwan":"Taiwan*",
"United States":"US", "Viet Nam":"Vietnam"}

arrive["location"]=[rd[xx] if xx in rd.keys() else xx for xx in arrive["location"].to_list()]
hh=list(arrive["location"].unique())

land=pd.read_csv("arrivals_land_definition_2.csv")
land=land[["date", "location", "land"]]
land=land[land["location"]!="Jordan"]
gg={"Lybia":"Libya",
   "UAE":"United Arab Emirates",
   "UK":"United Kingdom",
   "USA":"US"}

land["location"]=[gg[xx] if xx in gg.keys() else xx for xx in land["location"]]

#jj=list(land["location"].unique())
#np.sort([l for l in jj if l not in hh])
#np.sort([l for l in hh if l not in jj])
arrive=pd.merge(arrive, land, on=["date", "location"], how="outer")
arrive["air"][arrive["air"].isna()]=0
arrive["land"][arrive["land"].isna()]=0

pp={"Dominican Rep":"Dominican Republic",
   "Myanmar":"Burma"}

arrive["location"]=[pp[xx] if xx in pp.keys() else xx for xx in arrive["location"]]


arrive["num_arrivals"]=arrive["air"]+arrive["land"]
arrive.to_csv("arrivals_combined_conservative.csv")

In [40]:
##travel data cannot be released without permission

arrive=pd.read_csv("arrivals_air.csv")


rd={"Antigua-Barbuda":"Antigua and Barbuda", "Bosnia-Herzegovina":"Bosnia and Herzegovina", 
    "Brunei Darussalam":"Brunei",
"Central African Rep":"Central African Republic", "Cote D'Ivoire":"Cote d'Ivoire", 
    "Dominican Rep":"Dominican Rep", 
"Korea (South)":"Korea, South", "Macedonia":"North Macedonia", 
    "St Kitts-Nevis":"Saint Kitts and Nevis", "Taiwan":"Taiwan*",
"United States":"US", "Viet Nam":"Vietnam"}

arrive["location"]=[rd[xx] if xx in rd.keys() else xx for xx in arrive["location"].to_list()]
hh=list(arrive["location"].unique())

land=pd.read_csv("arrivals_land_definition_1.csv")
land=land[["date", "location", "land"]]
land=land[land["location"]!="Jordan"]
gg={"Lybia":"Libya",
   "UAE":"United Arab Emirates",
   "UK":"United Kingdom",
   "USA":"US"}

land["location"]=[gg[xx] if xx in gg.keys() else xx for xx in land["location"]]

#jj=list(land["location"].unique())
#np.sort([l for l in jj if l not in hh])
#np.sort([l for l in hh if l not in jj])
arrive=pd.merge(arrive, land, on=["date", "location"], how="outer")
arrive["air"][arrive["air"].isna()]=0
arrive["land"][arrive["land"].isna()]=0

pp={"Dominican Rep":"Dominican Republic",
   "Myanmar":"Burma"}

arrive["location"]=[pp[xx] if xx in pp.keys() else xx for xx in arrive["location"]]


arrive["num_arrivals"]=arrive["air"]+arrive["land"]
arrive.to_csv("arrivals_combined_whack.csv")

In [42]:
arrive["month"]=arrive["date"].str.replace("(\d{4})-", "").str.replace("-(\d{2})", "")

In [44]:
overll_arrivaLs=arrive.groupby(["month", "location"])[["num_arrivals","air", "land"]].apply(sum).reset_index()
overll_arrivaLs.to_csv("arrivals_alles.csv", index=None)

In [ ]:
ll=list(MT["country"].unique())
hh=list(arrive["location"].unique())
[l for l in ll if l not in hh]
[l for l in hh if l not in ll]

In [11]:
mindate=max([MT["date"].min(), arrive["date"].min()])
maxdate=min([MT["date"].max(), arrive["date"].max()])

MT=MT.rename(columns={"country":"location"})

seed=pd.merge(MT, arrive, on=["date", "location" ], how='left')
seed["nr_seed"]=seed["seeder_proportion"]*seed["num_arrivals"]
seed=seed[(seed["date"] >= mindate) & (seed["date"] <= "2020-12-31")]

In [13]:
III=seed.groupby(["date", "location"]).agg({'nr_seed':"sum"}).reset_index()
EII.to_csv("III.csv")